In [1]:
#####################################
# Developed by Baptiste PICARD      #
# picard.baptiste@laposte.net       #
# Started the 21th of April 2020    #
# Resume :                          #
# Evolution of the prices.          # 
# Model to predict the evolutions   #
#                                   #
#####################################

# Modules to import 

# Utils 
import time 
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pre-processing 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Models : classification.
# import xgboost as xgb # X Gradient Boosting (most powerfull)
# from sklearn.linear_model import LogisticRegression # Logistic Regression -> Problem isn't linear separable but -> benchmark
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier # Gradient Boosting, Random Forest


# Environment 
pd.set_option('display.max_columns', 50)
GRAFS = True
OUTLIERS = True

# Constants 
train_set = './data/sales_train.csv'
test_set = './data/test.csv'
subs_set = './data/sample_submission.csv'
items = './data/items.csv'
categories = './data/item_categories.csv'
shops = './data/shops.csv'

results = './data/results.txt'

In [16]:
# Open the .csv file  
df_train_set = pd.read_csv(train_set) # Read .csv file
df_test_set = pd.read_csv(test_set) # Read .csv file
df_subs_set = pd.read_csv(subs_set) # Read .csv file
df_shops_set = pd.read_csv(shops) # Read .csv file
df_items_set = pd.read_csv(items) # Read .csv file
df_categories_set = pd.read_csv(categories) # Read .csv file

In [40]:
df_train_set.head()


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [41]:
df_test_set.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [29]:
# Creating the df_train 
df_train_shop = pd.merge(df_train_set, df_shops_set, on='shop_id', how='inner')
df_train_shop_item = pd.merge(df_train_shop, df_items_set, on='item_id', how='inner')
df_train = pd.merge(df_train_shop_item, df_categories_set, on='item_category_id', how='inner')
df_train = df_train.drop(['shop_id', 'item_id', 'item_category_id'], axis=1)

# Creating the df_test / df_subs
df_test_shop = pd.merge(df_test_set, df_shops_set, on='shop_id', how='inner')
df_test_shop_items = pd.merge(df_test_shop, df_items_set, on='item_id', how='inner')
df_test = pd.merge(df_test_shop_items, df_categories_set, on='item_category_id', how='inner')
df_test = df_test.drop(['shop_id', 'item_id', 'item_category_id', 'ID'], axis=1)

# Train 
df_train_rows, df_train_columns = df_train.shape
# Test 
df_test_rows, df_test_columns = df_test.shape
print('Train set : \n train.csv : {} (rows/columns).'.format(df_train.shape))
print('Test set : \n test.csv : {} (rows/columns).'.format(df_test.shape))
print('Total nan values (train) : {}'.format(df_train.isnull().sum().sum()))
print('Total nan values (test) : {}'.format(df_test.isnull().sum().sum()))
df_train = df_train.rename(columns={'shop_name' : 'shop_name (RUSSIA)', 'item_name' : 'item_name (RUSSIA)', 'item_category_name': 'item_category_name (RUSSIA)'})
df_test = df_test.rename(columns={'shop_name' : 'shop_name (RUSSIA)', 'item_name' : 'item_name (RUSSIA)', 'item_category_name': 'item_category_name (RUSSIA)'})

Train set : 
 train.csv : (2935849, 7) (rows/columns).
Test set : 
 test.csv : (214200, 3) (rows/columns).
Total nan values (train) : 0
Total nan values (test) : 0


In [30]:
df_train.head()

,date,date_block_num,item_price,item_cnt_day,shop_name (RUSSIA),item_name (RUSSIA),item_category_name (RUSSIA)
0,02.01.2013,0,999.0,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray
1,02.01.2013,0,999.0,1.0,"Москва ТРК ""Атриум""",ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray
2,03.01.2013,0,999.0,1.0,"Москва ТРК ""Атриум""",ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray
3,20.01.2013,0,999.0,1.0,"Москва ТРК ""Атриум""",ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray
4,23.01.2013,0,999.0,1.0,"Москва ТРК ""Атриум""",ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray


In [31]:
df_test.head()

,shop_name (RUSSIA),item_name (RUSSIA),item_category_name (RUSSIA)
0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",Игры - PS3
1,"Волжский ТЦ ""Волга Молл""","NHL 15 [PS3, русские субтитры]",Игры - PS3
2,"Воронеж (Плехановская, 13)","NHL 15 [PS3, русские субтитры]",Игры - PS3
3,"Балашиха ТРК ""Октябрь-Киномир""","NHL 15 [PS3, русские субтитры]",Игры - PS3
4,"Адыгея ТЦ ""Мега""","NHL 15 [PS3, русские субтитры]",Игры - PS3


In [33]:
# Feature engineering -> No nan values.
for index, c_name in enumerate(df_train.columns) :
    print("{}). {}".format(index, c_name))
    print("Type : {}".format(df_train[c_name].dtype))
    print("Nan count : {}".format(df_train[c_name].isnull().sum()))
    print("Unique values : {}".format(len(df_train[c_name].unique())))
    # if(c_name=='date') :
    #     print("Encoding {}".format(c_name))
    #     df_train[c_name] = (pd.to_datetime(df_train[c_name]) - pd.datetime.now()).dt.total_seconds()
    #     enc = MinMaxScaler()
    #     df_train[c_name] = enc.fit_transform(df_train[c_name].values.reshape(-1, 1))
    # elif(c_name=="date_block_num") :
    #     print("Encoding {}".format(c_name))
    #     enc = LabelEncoder()
    #     df_train[c_name] = enc.fit_transform(df_train[c_name])
    # elif(c_name=="item_price") :
    #     print("Encoding {}".format(c_name))
    #     enc = MinMaxScaler()
    #     df_train[c_name] = enc.fit_transform(df_train[c_name].values.reshape(-1, 1))
    # elif(c_name=="item_cnt_day") :
    #     print("Encoding {}".format(c_name))
    #     enc = MinMaxScaler()
    #     df_train[c_name] = enc.fit_transform(df_train[c_name].values.reshape(-1, 1))
    # elif(c_name=="a") : 
    #     pass
        

0). date
Type : object
Nan count : 0
Unique values : 1034
1). date_block_num
Type : int64
Nan count : 0
Unique values : 34
2). item_price
Type : float64
Nan count : 0
Unique values : 19993
3). item_cnt_day
Type : float64
Nan count : 0
Unique values : 198
4). shop_name (RUSSIA)
Type : object
Nan count : 0
Unique values : 60
5). item_name (RUSSIA)
Type : object
Nan count : 0
Unique values : 21807
6). item_category_name (RUSSIA)
Type : object
Nan count : 0
Unique values : 84


In [39]:
a = df_train['item_category_name (RUSSIA)'].unique()
for item in df_test['item_category_name (RUSSIA)'].unique() :
    if(item not in a) :
        print(item)

In [ ]:
plt.figure()
plt.title('Categories')
ax = sns.countplot(df_train['item_category_name (RUSSIA)'])
plt.show()

In [37]:
shops = list(map(
    lambda x : {
                'item' : x['shop_name'],
                'city' : x['shop_name'].split(' ')[0].replace('"', ''),
                'type' : x['shop_name'].split(' ')[1] if(len(x['shop_name'].split(' '))>=3) else "Other",
                'name' : re.search(r'"([^;]*)"', x['shop_name'][1:-1]).group(1)[1:-1] 
                            if(x['shop_name'][0]=='"' and x['shop_name'][-1]=='"') 
                            else 'No name'
                },
    shop_name_train
    ))

items = {        
        # Cinema
        'Кино - Blu-Ray' : {'field' : 'Cinema', 'category' : 'Blu-ray'},
        'Кино - DVD' : {'field' : 'Cinema', 'category' : 'DVD'},        
        'Кино - Blu-Ray 3D' : {'field' : 'Cinema', 'category' : 'Blu-Ray 3D'},
        'Кино - Коллекционное' : {'field' : 'Cinema', 'category' : 'Collection'}, 
        'Кино - Blu-Ray 4K' : {'field' : 'Cinema', 'category' : 'Blu-Ray 4K'}, 

    
        # Accessories
        'Аксессуары - PSVita' : {'field' : 'Accessories', 'category' : 'PSVita'},
        'Аксессуары - PS3' : {'field' : 'Accessories', 'category' : 'PS3'},
        'Аксессуары - XBOX 360' : {'field' : 'Accessories', 'category' : 'XBOX 360'},
        'Аксессуары - PSP' : {'field' : 'Accessories', 'category' : 'PSP'},
        'Аксессуары - PS4' : {'field' : 'Accessories', 'category' : 'PS4'},
        'Аксессуары - XBOX ONE' : {'field' : 'Accessories', 'category' : 'XBOX ONE'},
        'PC - Гарнитуры/Наушники' : {'field' : 'Accessories', 'category' : 'Headphones'}, 
        'Аксессуары - PS2' : {'field' : 'Accessories', 'category' : 'PS2'},
        'Элементы питания' : {'field' : 'Accessories', 'category' : 'Battery'},
    
        # Music
        'Музыка - CD локального производства' : {'field' : 'Music', 'category' : 'CD'},
        'Музыка - Музыкальное видео' : {'field' : 'Music', 'category' : 'Clip video'},
        'Музыка - CD фирменного производства' : {'field' : 'Music', 'category' : 'CD'},
        'Музыка - MP3' : {'field' : 'Music', 'category' : 'MP3'}, 
        'Музыка - Подарочные издания' : {'field' : 'Music', 'category' : 'Gift Edition'},
        'Музыка - Винил' : {'field' : 'Music', 'category' : 'Vinyl'},
    
        # Games 
        'Игры - PSVita' : {'field' : 'Games', 'category' : 'PSVita'},
        'Игры - XBOX 360' : {'field' : 'Games', 'category' : 'XBOX 360'},
        'Игры - PSP' : {'field' : 'Games', 'category' : 'PSP'},
        'Игры PC - Стандартные издания' : {'field' : 'Games', 'category' : 'Pc Standard Editions'},
        'Игры - PS3' : {'field' : 'Games', 'category' : 'PS3'},
        'Игры PC - Дополнительные издания' : {'field' : 'Games', 'category' : 'Pc DLC'},
        'Игры PC - Коллекционные издания' : {'field' : 'Games', 'category' : 'Pc Collector'},
        'Игры - Аксессуары для игр' : {'field' : 'Games', 'category' : 'Accessories'},
        'Игры - PS4' : {'field' : 'Games', 'category' : 'PS4'}, 
        'Игры - XBOX ONE' : {'field' : 'Games', 'category' : 'XBOX ONE'}, 
        'Игры - PS2' : {'field' : 'Games', 'category' : 'PS2'},
        'Игры PC - Цифра' : {'field' : 'Games', 'category' : 'Digit'},
        'Игры Android - Цифра' : {'field' : 'Games', 'category' : 'Android'}, 
        'Игры MAC - Цифра' : {'field' : 'Games', 'category' : 'MAC'},
        
        # Gifts
        'Подарки - Гаджеты, роботы, спорт' : {'field' : 'Gifts', 'category' : 'Gadjet'},
        'Подарки - Мягкие игрушки' : {'field' : 'Gifts', 'category' : 'Toys'},        
        'Подарки - Сувениры' : {'field' : 'Gifts', 'category' : 'Souvenirs'},
        'Подарки - Настольные игры' : {'field' : 'Gifts', 'category' : 'Board games'},
        'Подарки - Настольные игры (компактные)' : {'field' : 'Gifts', 'category' : 'Board games'},
        'Подарки - Развитие' : {'field' : 'Gifts', 'category' : 'Développement'},
        'Подарки - Фигурки' : {'field' : 'Gifts', 'category' : 'Figurines'},
        'Подарки - Сувениры (в навеску)' : {'field' : 'Gifts', 'category' : 'Souvenirs'},
        'Подарки - Сумки, Альбомы, Коврики д/мыши' : {'field' : 'Gifts', 'category' : 'Bags, Albums, Mousepads'},
        'Подарки - Атрибутика' : {'field' : 'Gifts', 'category' : 'Attributes'},
        'Подарки - Открытки, наклейки' : {'field' : 'Gifts', 'category' : 'Postcards / Stickers'},
        'Подарки - Сертификаты, услуги' : {'field' : 'Gifts', 'category' : 'Certificates / Services'},
    
        # Payement Cards
        'Карты оплаты - PSN' : {'field' : 'Payement Cards', 'category' : 'PSN'},
        'Карты оплаты - Live!' : {'field' : 'Payement Cards', 'category' : 'Live!'},
        'Карты оплаты (Кино, Музыка, Игры)' : {'field' : 'Payement Cards', 'category' : 'Cinema / Music / Games'},
        'Карты оплаты - Live! (Цифра)' : {'field' : 'Payement Cards', 'category' : 'Live!'},
        'Карты оплаты - Windows (Цифра)' : {'field' : 'Payement Cards', 'category' : 'Windows'},
    
        # Programs
        'Программы - Для дома и офиса' : {'field' : 'Programs', 'category' : 'Home/Office'},
        'Программы - 1С:Предприятие 8' : {'field' : 'Programs', 'category' : 'Entreprise'},
        'Служебные' : {'field' : 'Programs', 'category' : 'Office'},
        'Программы - Обучающие' : {'field' : 'Programs', 'category' : 'Education'},
        'Программы - Для дома и офиса (Цифра)' : {'field' : 'Programs', 'category' : 'Home/Office'},
        'Программы - Обучающие (Цифра)' : {'field' : 'Programs', 'category' : 'Education'},
        'Программы - MAC (Цифра)' : {'field' : 'Programs', 'category' : 'MAC'},
    
        # Console
        'Игровые консоли - XBOX 360' : {'field' : 'Consoles', 'category' : 'XBOX 360'},
        'Игровые консоли - PSP' : {'field' : 'Consoles', 'category' : 'PSP'},  
        'Игровые консоли - PS3' : {'field' : 'Consoles', 'category' : 'PS3'},
        'Игровые консоли - PSVita' : {'field' : 'Consoles', 'category' : 'PSVita'},
        'Игровые консоли - Прочие' : {'field' : 'Consoles', 'category' : 'Other'}, 
        'Игровые консоли - XBOX ONE' : {'field' : 'Consoles', 'category' : 'XBOX ONE'},
        'Игровые консоли - PS4' : {'field' : 'Consoles', 'category' : 'PS4'},
        'Игровые консоли - PS2' : {'field' : 'Consoles', 'category' : 'PS2'},
    
        # Books
        'Книги - Методические материалы 1С' : {'field' : 'Books', 'category' : 'Methods'},
        'Книги - Аудиокниги' : {'field' : 'Books', 'category' : 'Audiobooks'},
        'Книги - Аудиокниги 1С' : {'field' : 'Books', 'category' : 'Audiobooks'},
        'Книги - Комиксы, манга' : {'field' : 'Books', 'category' : 'Manga / Comics'},
        'Книги - Артбуки, энциклопедии' : {'field' : 'Books', 'category' : 'Artbooks / Encyclopedias'},
        'Книги - Бизнес литература' : {'field' : 'Books', 'category' : 'Business Literature'},
        'Книги - Путеводители' : {'field' : 'Books', 'category' : 'Guides'},
        'Книги - Художественная литература' : {'field' : 'Books', 'category' : 'Fiction'},
        'Книги - Открытки' : {'field' : 'Books', 'category' : 'Postcards'},
        'Книги - Компьютерная литература' : {'field' : 'Books', 'category' : 'Computer literature'}, 
        'Книги - Познавательная литература' : {'field' : 'Books', 'category' : 'Education'}, 
        'Книги - Цифра' : {'field' : 'Books', 'category' : 'Digit'},
        'Книги - Аудиокниги (Цифра)' : {'field' : 'Books', 'category' : 'Audiobooks'},
    
        # Services
        'Билеты (Цифра)' : {'field' : 'Services', 'category' : 'Tickets'},
        'Служебные - Билеты' : {'field' : 'Services', 'category' : 'Tickets'},
        'Доставка товара' : {'field' : 'Services', 'category' : 'Delivery'},
    
        # Media
        'Чистые носители (штучные)' : {'field' : 'Media', 'category' : 'Piece'},
        'Чистые носители (шпиль)' : {'field' : 'Media', 'category' : 'Spire'},
}

fields = []
categories = []
for key in items.keys() :
    if(items[key]['field'] not in fields) :
        fields.append(items[key]['field'])
    if(items[key]['category'] not in categories) :
        categories.append(items[key]['category'])

print(fields)
print(categories)

['Cinema', 'Accessories', 'Music', 'Games', 'Gifts', 'Payement Cards', 'Programs', 'Consoles', 'Books', 'Services', 'Media']
['Blu-ray', 'DVD', 'Blu-Ray 3D', 'Collection', 'Blu-Ray 4K', 'PSVita', 'PS3', 'XBOX 360', 'PSP', 'PS4', 'XBOX ONE', 'Headphones', 'PS2', 'Battery', 'CD', 'Clip video', 'MP3', 'Gift Edition', 'Vinyl', 'Pc Standard Editions', 'Pc DLC', 'Pc Collector', 'Accessories', 'Digit', 'Android', 'MAC', 'Gadjet', 'Toys', 'Souvenirs', 'Board games', 'Développement', 'Figurines', 'Bags, Albums, Mousepads', 'Attributes', 'Postcards / Stickers', 'Certificates / Services', 'PSN', 'Live!', 'Cinema / Music / Games', 'Windows', 'Home/Office', 'Entreprise', 'Office', 'Education', 'Other', 'Methods', 'Audiobooks', 'Manga / Comics', 'Artbooks / Encyclopedias', 'Business Literature', 'Guides', 'Fiction', 'Postcards', 'Computer literature', 'Tickets', 'Delivery', 'Piece', 'Spire']


In [ ]:
print(df_train['shop_name (RUSSIA)'].unique())
#'Москва ТК "Буденовский" (пав.К7)' = 'Москва ТК "Буденовский"'
# 'Москва ТК "Буденовский" (пав.А2)' = 'Москва ТК "Буденовский"'

In [ ]:
it = {
        'Ярославль ТЦ "Альтаир"' : {'name' : 'Yaroslavl', 'type' : 'Commercial center', 'location' : "Moscow"}, 
        'Москва ТРК "Атриум"' : {'name' : 'Atrium', 'type' : 'TRC', 'location' : "Moscow"}, 
        'Москва ТК "Буденовский" (пав.К7)' : {'name' : 'Budenovski', 'type' : 'TC', 'location' : "Moscow"}, 
        'Москва ТК "Буденовский" (пав.А2)' : {'name' : 'Budenovski', 'type' : 'TC', 'location' : "Moscow"}, 
        'Москва МТРЦ "Афи Молл"' : {'name' : 'Afi Mall', 'type' : 'MTC', 'location' : "Moscow"}, 
        'Москва ТЦ "МЕГА Теплый Стан" II' : {'name' : 'MEGA Teply Stan II', 'type' : 'TC', 'location' : "Moscow"}, 
        'Москва ТЦ "МЕГА Белая Дача II"' : {'name' : 'MEGA Belaya Dacha II', 'type' : 'TC', 'location' : "Moscow"}, 
        'Воронеж (Плехановская, 13)' : {'name' : 'Plekhanovskaya', 'type' : 'Other', 'location' : "Voronezh"}, 
        'Воронеж ТРЦ "Максимир"' : {'name' : 'Plekhanovskaya', 'type' : 'SEC', 'location' : "Maximir"}, 
        '!Якутск Орджоникидзе, 56 фран' : {'name' : 'Plekhanovskaya', 'type' : 'SEC', 'location' : "Maximir"}, 
        '!Якутск ТЦ "Центральный" фран'
        'Коломна ТЦ "Рио"'
        'Калуга ТРЦ "XXI век"'
        'Москва ТЦ "Семеновский"'
        'Н.Новгород ТРЦ "Фантастика"'
        'Чехов ТРЦ "Карнавал"'
        'Химки ТЦ "Мега"'
        'Сургут ТРЦ "Сити Молл"'
        'СПб ТК "Невский Центр"'
        'Тюмень ТЦ "Зеленый Берег"'
        'Омск ТЦ "Мега"'
        'Самара ТЦ "Мелодия"'
        'Новосибирск ТЦ "Мега"'
        'Сергиев Посад ТЦ "7Я"'
        'Якутск Орджоникидзе, 56'
        'Якутск ТЦ "Центральный"'
        'Курск ТЦ "Пушкинский"'
        'Москва Магазин С21'
        'Красноярск ТЦ "Июнь"'
        'Москва ТЦ "Новый век" (Новокосино)'
        'Москва ТЦ "Ареал" (Беляево)'
        'Волжский ТЦ "Волга Молл"'
        'Адыгея ТЦ "Мега"'
        'Воронеж ТРЦ Сити-Парк "Град"'
        'Жуковский ул. Чкалова 39м?'
        'Казань ТЦ "ПаркХаус" II'
        'Казань ТЦ "Бехетле"'
        'Интернет-магазин ЧС'
        'Уфа ТЦ "Семья" 2'
        'Москва ТЦ "Перловский"' 
        'Москва ТЦ "Серебряный Дом"'
        'Тюмень ТЦ "Гудвин"'
        'СПб ТК "Сенная"'
        'Уфа ТК "Центральный"'
        'РостовНаДону ТЦ "Мега"'
        'Самара ТЦ "ПаркХаус"'
        'Вологда ТРЦ "Мармелад"'
        'Красноярск ТЦ "Взлетка Плаза"'
        'Тюмень ТРЦ "Кристалл"'
        'РостовНаДону ТРК "Мегацентр Горизонт"'
        'Томск ТРЦ "Изумрудный Город"'
        'Н.Новгород ТРЦ "РИО"'
        'Мытищи ТРК "XL-3"'
        'Балашиха ТРК "Октябрь-Киномир"'
        'Жуковский ул. Чкалова 39м²'
        'Новосибирск ТРЦ "Галерея Новосибирск"'
        'Выездная Торговля'
        'РостовНаДону ТРК "Мегацентр Горизонт" Островной'
        'Москва "Распродажа"'
        'Цифровой склад 1С-Онлайн'
     }

In [ ]:
df_train['number_words_item_category'] = len(df_train['item_category_name (RUSSIA)'].str.split('\s+'))

In [ ]:
df_train.head()

In [ ]:
%%time
cmpt2 = 0
cmpt = []
for index, row in df_train.iterrows(): 
    copy = row.copy()
    max_len = len(copy['item_category_name (RUSSIA)'].split('-'))
    if(max_len==2):
        cmpt2 +=1
    else :
        cmpt.append({'index' : index, "splitlen" : max_len})

In [ ]:
{'item': 'Выездная Торговля', 'city': 'Выездная', 'type': 'Торговля', 'name': 'Other'}
{'item': 'Москва Магазин С21', 'city': 'Москва', 'type': 'Магазин', 'name': 'No name'}


def getCity(col) :
    return col.split(' ')[0].replace('"', '') 

def getType(col) :
    l = len(col.split(' '))
    if(l >= 3) :
        return col.split(' ')[1]
    else : 
        return 'Other'
        
def getAdress(col) :
    if(col[0]=='"' and col[-1]=='"')  :
        return re.search(r'"([^;]*)"', col[1:-1]).group(1)[1:-1] 
    else :
        return 'No name'
        *
        

for it in shops : 
    if(it['city'] == '!Якутск'):
        it['city'] = 'Якутск'
    if(it['city'] == 'Сергиев'):
        it['city'] = 'СергиевПосад'
        it['type'] = 'ТЦ'
        
    if(it['item'] == 'Интернет-магазин ЧС') :
        it['type'] = 'Other'
        it['name'] = 'No ЧС'
    if(it['item'] ==  'Цифровой склад 1С-Онлайн') :
        it['type'] = 'склад'
        it['name'] = '1С-Онлайн'
    if(it['item'] ==  'Якутск Орджоникидзе, 56' or it['item'] ==  '!Якутск Орджоникидзе, 56 фран') :
        it['type'] = 'Орджоникидзе'
        it['name'] = '56'
    if(it['item'] ==  'Жуковский ул. Чкалова 39м²' or it['item'] ==  'Жуковский ул. Чкалова 39м?') :
        it['city'] = 'Жуковский ул.'
        it['type'] = 'Чкалова'
        it['name'] = '39м'
    if(it['item'] ==  'Воронеж (Плехановская, 13)') :
        it['type'] = 'Плехановская'
        it['name'] = '13'
    if(it['item'] ==  'Выездная Торговля') :
        it['type'] = 'Торговля'
        it['name'] = 'No name'
    if(it['item'] ==  'Москва Магазин С21') :
        it['type'] = 'Магазин'
        it['name'] = 'С21'